In [18]:
import warnings
warnings.filterwarnings(action="ignore")
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report, confusion_matrix

In [19]:
window_size = 2040
n_components = 30

In [20]:
# Cargar las variables independientes (X_train) sin cabecera desde archivo
X_train = np.loadtxt(f'../data/v7/transformed_train_data_comp_{n_components}.csv', delimiter=',')

# Cargar las variables dependientes (y_train) con cabecera desde archivo
y_train = pd.read_csv(f'../data/train_value_min_label_windows_{window_size}_llm.csv')

# Cargar el conjunto de test (X_test sin cabeceras y y_test con cabeceras)
X_test = np.loadtxt(f'../data/v7/transformed_test_data_comp_{n_components}.csv', delimiter=',')
y_test = pd.read_csv(f'../data/test_value_min_label_windows_{window_size}_llm.csv')

# Eliminar la columna "row" que es solo un índice
y_train = y_train.drop(columns=['row'])
y_test = y_test.drop(columns=['row'])


In [21]:
# Escalar los datos de entrenamiento y test
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(X_train_scaled.shape, X_test_scaled.shape)



(1310, 2) (586, 2)


In [23]:
from xgboost import XGBClassifier

# Bucle para realizar clasificación binaria para cada columna (clase)
for col in y_train.columns:
    y_train_bin = y_train[col]
    y_test_bin = y_test[col]
    
    # Verificar que haya más de una clase en el conjunto de entrenamiento
    if len(np.unique(y_test_bin)) > 1:
        # Calcular la proporción de clases
        neg_count = np.sum(y_train_bin == 0)
        pos_count = np.sum(y_train_bin == 1)
        
        # Ajustar el peso para desbalanceo de clases
        scale_pos_weight = neg_count / pos_count if pos_count > 0 else 1
        
        # Entrenar modelo XGBoost
        xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss', scale_pos_weight=scale_pos_weight)
        xgb_clf.fit(X_train_scaled, y_train_bin)
    
        # Predecir en el conjunto de test
        y_pred_bin = xgb_clf.predict(X_test_scaled)
        y_pred_proba = xgb_clf.predict_proba(X_test_scaled)[:, 1]  # Para calcular AUC
    
        # Calcular métricas
        accuracy = accuracy_score(y_test_bin, y_pred_bin)
        f1 = f1_score(y_test_bin, y_pred_bin)
        auc = roc_auc_score(y_test_bin, y_pred_proba)
        report = classification_report(y_test_bin, y_pred_bin)
        cm = confusion_matrix(y_test_bin, y_pred_bin)
    
        # Mostrar resultados
        print(f"Resultados para la clase {col}:")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"F1 Score: {f1:.4f}")
        print(f"AUC: {auc:.4f}")
        print("Classification Report:\n", report)
        print(f"Confusion Matrix:\n{cm}")
    else:
        # En caso de tener solo una clase
        accuracy = accuracy_score(y_test_bin, np.full_like(y_test_bin, y_train_bin.iloc[0]))
        
        # Mostrar solo accuracy, el resto no se puede calcular
        print(f"Resultados para la clase {col}:")
        print(f"Accuracy (una sola clase): {accuracy:.4f}")
        print("F1 Score: No calculable (una sola clase)")
        print("AUC: No calculable (una sola clase)")

c:\Users\DennysMallqui\anaconda3\envs\py311\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:29:47] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\DennysMallqui\anaconda3\envs\py311\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:29:47] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\DennysMallqui\anaconda3\envs\py311\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:29:47] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg,

Resultados para la clase Sockets01:
Accuracy: 0.9795
F1 Score: 0.1429
AUC: 0.8520
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       578
           1       0.17      0.12      0.14         8

    accuracy                           0.98       586
   macro avg       0.58      0.56      0.57       586
weighted avg       0.98      0.98      0.98       586

Confusion Matrix:
[[573   5]
 [  7   1]]
Resultados para la clase Sockets02:
Accuracy (una sola clase): 1.0000
F1 Score: No calculable (una sola clase)
AUC: No calculable (una sola clase)
Resultados para la clase Light01:
Accuracy (una sola clase): 1.0000
F1 Score: No calculable (una sola clase)
AUC: No calculable (una sola clase)
Resultados para la clase CE appliance01:
Accuracy: 0.9283
F1 Score: 0.9302
AUC: 0.9395
Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.99      0.93       268
           1  

c:\Users\DennysMallqui\anaconda3\envs\py311\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:29:47] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\DennysMallqui\anaconda3\envs\py311\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:29:47] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\DennysMallqui\anaconda3\envs\py311\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:29:47] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg,

Resultados para la clase Light02:
Accuracy: 0.7491
F1 Score: 0.3691
AUC: 0.7801
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.86      0.84       460
           1       0.40      0.34      0.37       126

    accuracy                           0.75       586
   macro avg       0.61      0.60      0.61       586
weighted avg       0.74      0.75      0.74       586

Confusion Matrix:
[[396  64]
 [ 83  43]]
Resultados para la clase Sockets03:
Accuracy: 0.8942
F1 Score: 0.0312
AUC: 0.7311
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.90      0.94       579
           1       0.02      0.14      0.03         7

    accuracy                           0.89       586
   macro avg       0.50      0.52      0.49       586
weighted avg       0.98      0.89      0.93       586

Confusion Matrix:
[[523  56]
 [  6   1]]
Resultados para la clase Light03:
Accuracy (una sola c

c:\Users\DennysMallqui\anaconda3\envs\py311\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:29:47] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\DennysMallqui\anaconda3\envs\py311\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:29:47] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\DennysMallqui\anaconda3\envs\py311\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:29:47] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg,

Resultados para la clase Light05:
Accuracy: 0.8003
F1 Score: 0.3390
AUC: 0.7936
Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.86      0.88       512
           1       0.29      0.41      0.34        74

    accuracy                           0.80       586
   macro avg       0.60      0.63      0.61       586
weighted avg       0.83      0.80      0.81       586

Confusion Matrix:
[[439  73]
 [ 44  30]]
Resultados para la clase Unknown01:
Accuracy: 0.9795
F1 Score: 0.0000
AUC: 0.2774
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.98      0.99       584
           1       0.00      0.00      0.00         2

    accuracy                           0.98       586
   macro avg       0.50      0.49      0.49       586
weighted avg       0.99      0.98      0.99       586

Confusion Matrix:
[[574  10]
 [  2   0]]
Resultados para la clase Sockets04:
Accuracy (una sola

c:\Users\DennysMallqui\anaconda3\envs\py311\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:29:47] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\DennysMallqui\anaconda3\envs\py311\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:29:47] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Resultados para la clase Washer dryer02:
Accuracy: 0.9590
F1 Score: 0.2000
AUC: 0.8325
Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.98      0.98       570
           1       0.21      0.19      0.20        16

    accuracy                           0.96       586
   macro avg       0.60      0.58      0.59       586
weighted avg       0.96      0.96      0.96       586

Confusion Matrix:
[[559  11]
 [ 13   3]]
